In [2]:
import sys
import os
sys.path.append(os.path.abspath('./../../..'))
from pipeline.lib.UrlGenerator import UrlGenerator
from Controllers.MarkedCellController import MarkedCellController,get_DataFrame_from_query_result
from cell_extractor.CellDetector import MultiThresholdDetector
from cell_extractor.CellAnnotationUtilities import CellAnnotationUtilities
import numpy as np
urlGen = UrlGenerator()

In [3]:
controller = MarkedCellController()
animali = 'DK41'
search_dict = {'FK_prep_id':animali,'FK_cell_type_id':17}
round3_unsures2000 = controller.get_marked_cells(search_dict)
search_dict = {'FK_prep_id':animali,'FK_cell_type_id':16}
round3_sures2000 = controller.get_marked_cells(search_dict)

In [4]:
round3_unsures2000 = get_DataFrame_from_query_result(round3_unsures2000,'Unsure2000',[0.325,0.325,20])
round3_sures2000 = get_DataFrame_from_query_result(round3_sures2000,'Sure2000',[0.325,0.325,20])
round3_unsures2000 = round3_unsures2000[['NULL' not in i for i in round3_unsures2000.name]]
round3_sures2000 = round3_sures2000[['NULL' not in i for i in round3_sures2000.name]]

In [5]:
marissa_unsures2000 = round3_unsures2000[['Marissa' in i for i in  round3_unsures2000.name]]
marissa_sures2000 = round3_sures2000[['Marissa' in i for i in  round3_sures2000.name]]
julian_unsures2000 = round3_unsures2000[['Julian' in i for i in  round3_unsures2000.name]]

In [6]:
animali = 'DK52'
detector = MultiThresholdDetector(animali,round = 2)
sures = detector.get_sures()
sures = sures[['2000' in i for i in sures.name]]
unsures = detector.get_unsures()
unsures = unsures[['2000' in i for i in unsures.name]]

In [7]:
sure_points = np.array([sures.x,sures.y,sures.section]).T.astype(float)
unsure_points = np.array([unsures.x,unsures.y,unsures.section]).T.astype(float)
marissa_unsures2000_points = np.array([marissa_unsures2000.x,marissa_unsures2000.y,marissa_unsures2000.section]).T.astype(float)
marissa_sures2000_points = np.array([marissa_sures2000.x,marissa_sures2000.y,marissa_sures2000.section]).T.astype(float)
julian_unsures2000_points = np.array([julian_unsures2000.x,julian_unsures2000.y,julian_unsures2000.section]).T.astype(float)
util = CellAnnotationUtilities()
marissa_unsure_ids = util.find_cloest_neighbor_among_points(unsure_points,marissa_unsures2000_points,max_distance=0.5)
marissa_sure_ids = util.find_cloest_neighbor_among_points(sure_points,marissa_sures2000_points,max_distance=0.5)
julian_unsure_ids = util.find_cloest_neighbor_among_points(unsure_points,julian_unsures2000_points,max_distance=0.5)

skipping, min distance 66.1265149265505
skipping, min distance 261.25102983857204
skipping, min distance 151.9628769678111
skipping, min distance 1821.2149960329089
skipping, min distance 5149.3324735652495
skipping, min distance 1997.320204674267
skipping, min distance 3961.3733348707433
skipping, min distance 2814.4593729756034
skipping, min distance 1370.0
skipping, min distance 511.53848467321643
skipping, min distance 4266.0819468822765
skipping, min distance 206.30490504627244
skipping, min distance 1204.1484162195372
skipping, min distance 4271.240626001675
skipping, min distance 66.1265149265505
skipping, min distance 261.25102983857204
skipping, min distance 151.9628769678111
skipping, min distance 1821.2149960329089
skipping, min distance 5149.3324735652495
skipping, min distance 3961.3733348707433
skipping, min distance 1997.320204674267
skipping, min distance 1370.0
skipping, min distance 2814.4593729756034
skipping, min distance 511.53848467321643


In [9]:
#restore julian
urlGen.load_database_url(621)
annotated = np.floor(unsure_points[julian_unsure_ids])
double_anno = 0
done = 0
for i in urlGen.layers[-1]['annotations']:
    point = np.floor(i['point'])
    if np.any([np.all(point ==i) for i in annotated]):
        if 'description' in i:
            double_anno+=1
        else:
            done+=1
            try:
                original_id = np.where([np.all(np.logical_and((i[:2]-point[:2])<=1,i[2]==point[2])) for i in np.floor(julian_unsures2000_points).astype(int)])[0][0]
                original_annotation = julian_unsures2000.iloc[original_id]['name'].split('_')[-1].lower()
                i['description'] = original_annotation
            except:
                print(original_annotation)
model = urlGen.controller.get_urlModel(126)
model.url = urlGen.get_url()
urlGen.controller.update_row(model)

In [37]:
[i['name'] for i in urlGen.layers]

['DK52', 'DK52', 'DK52', 'Sure', 'Unsure']

In [24]:
#restore marissa
#unsure
urlGen.load_database_url(623)
annotated = np.floor(unsure_points[marissa_unsure_ids])
double_anno = 0
done = 0
for i in urlGen.layers[-1]['annotations']:
    point = np.floor(i['point'])
    if np.any([np.all(point ==i) for i in annotated]):
        if 'description' in i:

            double_anno+=1
        else:
            try:
                original_id = np.where([np.all(np.logical_and((i[:2]-point[:2])<=1,i[2]==point[2])) for i in np.floor(marissa_unsures2000_points).astype(int)])[0][0]
                original_annotation = marissa_unsures2000.iloc[original_id]['name'].split('_')[-1].lower()
                i['description'] = original_annotation
                done+=1
            except:
                print(point)
# for i in urlGen.layers[-1]['annotations']:
#     if i['description']=='':
#         del i['description']
print(f'double annotation {double_anno}')
print(f'annotation changed {done}')
#sure
annotated = np.floor(sure_points[marissa_sure_ids])
double_anno = 0
done = 0
for i in urlGen.layers[-2]['annotations']:
    point = np.floor(i['point'])
    if np.any([np.all(point ==i) for i in annotated]):
        if 'description' in i:
            double_anno+=1
        else:
            try:
                original_id = np.where([np.all(np.logical_and((i[:2]-point[:2])<=1,i[2]==point[2])) for i in np.floor(marissa_sures2000_points).astype(int)])[0][0]
                original_annotation = marissa_sures2000.iloc[original_id]['name'].split('_')[-1].lower()
                i['description'] = original_annotation
                done+=1
            except:
                print(point)
# for i in urlGen.layers[-2]['annotations']:
#     if i['description']=='':
#         del i['description']
print(f'double annotation {double_anno}')
print(f'annotation changed {done}')

model = urlGen.controller.get_urlModel(164)
model.url = urlGen.get_url()
urlGen.controller.update_row(model)

double annotation 0
annotation changed 430
double annotation 0
annotation changed 2


In [26]:
for i in urlGen.layers[-1]['annotations']:
    if 'description' in i:
        if i['description']!='':
            print(i['description'])

negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
positive
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
positive
negative
negative
negative
negative
negative
positive
positive
positive
positive
negative
negative
negative
negative
negative
negative
negative
negative
negative
negative
positive
n

In [22]:
'description' in i

False

In [19]:
i['description']==''

KeyError: 'description'

In [14]:
original_annotation

'negative'